# Test embedalign with SentEval 

This notebook will allow you to test EmbedAlign using SentEval. In particular, this also works on **CPUs** :D

* Dependencies:
    * Python 3.5 with NumPy/SciPy
    * Pytorch 
    * Tensorflow 1.5.0  (for CPUs or GPUs depending on how you plan to run it)
        * For example in MacOS: 
        ```
        pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.5.0-py3-none-any.whl
        ```
    * scikit-learn>=0.18.0
    * dill>=0.2.7.1


* Install `dgm4nlp` by following the instructions [here](https://github.com/uva-slpl/dgm4nlp), we highly recommend the use of `virtualenv`.

In the same `virtualenv`, do the following:

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```

* Install senteval
```
    python setup.py install
```

* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 

```
    cd data/downstream/
    ./get_transfer_data.bash
```

* Download [pretained embedlaign model](https://surfdrive.surf.nl/files/index.php/s/9M4h5zqmYETSmf3)


* The following code evaluates embedalign pretrained embeddings on en-fr Europarl on different NLP downstream tasks.



In [1]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
#import data 
# data.py is part of Senteval and it is used for loading word2vec style files
import senteval
import tensorflow as tf
import logging
from collections import defaultdict
import dill
import dgm4nlp
import os

/home/tom/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# SkipGram imports
sys.path.append('../Practical 2/code/')
from models.skipgram import SkipGram
from helpers import load_model
import msgpack
import torch

In [3]:
import platform
platform.python_version()

'3.5.4'

In [4]:
class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class EmbeddingExtractor:
    """
    This will compute a forward pass with the inference model of EmbedAlign and 
        give you the variational mean for each L1 word in the batch.
        
    Note that this takes monolingual L1 sentences only (at this point we have a traiend EmbedAlign model
        which dispenses with L2 sentences).    
        
    You don't really want to touch anything in this class.
    """

    def __init__(self, graph_file, ckpt_path, config=None):        
        g1 = tf.Graph()
        self.meta_graph = graph_file
        self.ckpt_path = ckpt_path
        
        self.softmax_approximation = 'botev-batch' #default
        with g1.as_default():
            self.sess = tf.Session(config=config, graph=g1)
            # load architecture computational graph
            self.new_saver = tf.train.import_meta_graph(self.meta_graph)
            # restore checkpoint
            self.new_saver.restore(self.sess, self.ckpt_path) #tf.train.latest_checkpoint(
            self.graph = g1  #tf.get_default_graph()
            # retrieve input variable
            self.x = self.graph.get_tensor_by_name("X:0")
            # retrieve training switch variable (True:trianing, False:Test)
            self.training_phase = self.graph.get_tensor_by_name("training_phase:0")
            #self.keep_prob = self.graph.get_tensor_by_name("keep_prob:0")

    def get_z_embedding_batch(self, x_batch):
        """
        :param x_batch: is np array of shape [batch_size, longest_sentence] containing the unique ids of words
        
        :returns: [batch_size, longest_sentence, z_dim]        
        """
        # Retrieve embeddings from latent variable Z
        # we can sempale several n_samples, default 1
        try:
            z_mean = self.graph.get_tensor_by_name("z:0")
            
            feed_dict = {
                self.x: x_batch,
                self.training_phase: False,
                #self.keep_prob: 1.

            }
            z_rep_values = self.sess.run(z_mean, feed_dict=feed_dict) 
        except:
            raise ValueError('tensor Z not in graph!')
        return z_rep_values
    
def get_idf(batch):
    """
    Compute the idf of a batch of sentences, where we consider each sentence a 'document'.
    """
    df = defaultdict(lambda: 0)
    
    for sent in batch:
        for token in set(sent):
            df[token] += 1
    
    # Here we compute the batch-wise idf, log(1 + |batch| / |{s \in batch: w \in s}|), to tf-idf weigh sentence embeddings
    idf = {word: np.log(1 + len(batch) / float(frequency)) for word, frequency in df.items()}
    
    return idf
    

This is how you interface with SentEval. The only think you need to change are the paths to trained models in the main block at the end.

In [ ]:

# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tks1': None,
                   'idf': False}
params_senteval = dotdict(params_senteval)

# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}


def prepare(params, samples):
    """
    In this example we are going to load a tensorflow model, 
    we open a dictionary with the indices of tokens and the computation graph
    """
    params.extractor = EmbeddingExtractor(
        graph_file='%s.meta'%(params.ckpt_path),
        ckpt_path=params.ckpt_path,
        config=None #run in cpu
    )

    # load tokenizer from training
    params.tks1 = dill.load(open(params.tok_path, 'rb'))
    return


def batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
    # of the pre-trained EmbedAlign model
    # from tokens ot ids position 0 is en
    batch = [params.tks1[0].to_sequences([(' '.join(s))]) for s in batch]
    
    # The idf will be used to weigh embeddings when averaging
    if params.idf:
        idf = get_idf([list(s[0]) for s in batch])
        
    embeddings = []
    for x1 in batch:
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)

        # sentence vector is the mean of word embeddings in context
        # [1, z_dim]
        if params.idf:
            weights = np.array([[[idf[token]] for token in list(x1[0])]])
            sent_vec = np.sum(z_batch1 * weights, axis=1) / np.sum(weights, axis=1)
        else:
            sent_vec = np.mean(z_batch1, axis=1)
            
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings


# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    # Disable GPU, memory overflow error otherwise
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    # define paths
    # path to senteval data
    # note senteval adds downstream into the path
    params_senteval.task_path = '/home/tom/SentEval/data/' 
    # path to computation graph
    # we use best model on validation AER
    # TODO: you have to point to valid paths! Use the pre-trained model linked from the top of this notebook.
    params_senteval.ckpt_path = '/home/tom/Documents/ULL/Practical 3/data/ull-practical3-embedalign/model.best.validation.aer.ckpt'
    # path to tokenizer with ids of trained Europarl data
    # out dictionary id depends on dill for pickle
    params_senteval.tok_path = '/home/tom/Documents/ULL/Practical 3/data/ull-practical3-embedalign/tokenizer.pickle'
    # Whether to use idf
    params_senteval.idf = True
    
    # we use 10 fold cross validation
    params_senteval.kfold = 10
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-25 15:27:42,840 : ***** Transfer task : MR *****




INFO:tensorflow:Restoring parameters from /home/tom/Documents/ULL/Practical 3/data/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-25 15:27:43,589 : Restoring parameters from /home/tom/Documents/ULL/Practical 3/data/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-25 15:27:44,013 : Generating sentence embeddings
2018-05-25 15:28:26,312 : Generated sentence embeddings
2018-05-25 15:28:26,313 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-25 15:28:38,851 : Best param found at split 1: l2reg = 8                 with score 63.96
2018-05-25 15:28:51,929 : Best param found at split 2: l2reg = 8                 with score 64.07
2018-05-25 15:29:05,113 : Best param found at split 3: l2reg = 8                 with score 64.21
2018-05-25 15:29:17,908 : Best param found at split 4: l2reg = 8                 with score 64.5


In [5]:
def prepare(params, samples):
    """
    We load the PyTorch model and accompanying word_to_index tokenizer.
    """
    params.extractor = load_model(params.ckpt_path, params.extractor)
    params.tokenizer = msgpack.unpack(open(params.tok_path, 'rb'), encoding='utf-8')
    
def batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by SkipGram) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    # if a sentence is empty dot is set to be the only token. We also remove UNKS as they bear no info, and tokenize
#     batch = [sent if sent != [] else ['.'] for sent in batch]
    batch = [[params.tokenizer[w] for w in s if w in params.tokenizer] if s != [] else [params.tokenizer['.']] for s in batch]
    embeddings = []
    
    if params.idf:
        df = defaultdict(lambda: 0)
        # Here we compute the batch-wise idf, log(1 + |batch| / |{s \in batch: w \in s}|), to tf-idf weigh sentence embeddings
        for sent in batch:
            for token in set(sent):
                df[token] += 1
    
        idf = {word: np.log(len(batch) / float(frequency)) for word, frequency in df.items()}
    
    for sent in batch:
        # Convert sentence to torch.tensor if not empty, else we set it to a dot again. 
        x = torch.tensor(sent, dtype=torch.long, device=params.device).unsqueeze(0) if sent else torch.tensor([params.tokenizer['.']], dtype=torch.long, device=params.device).unsqueeze(0)
        print(x, sent)
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, d_dim]
        word_embeddings = params.extractor.lst_pass(x)[0].unsqueeze(0)
        
        # sentence vector is the mean of word embeddings in context
        # [1, d_dim]
        if params.idf:
            # [1, sentence_length, 1] tensor of idfs per word
            weights = torch.tensor([idf[token] for token in sent] if sent else [1.], device=params.device).unsqueeze(0).unsqueeze(2)
#             print(weights, sent)
            # multiply embeddings by their weights and sum, dividing by total weight, yielding tf-idf weighed sent embedding 
            sent_embedding = (torch.sum(weights * word_embeddings, dim=1) / torch.sum(weights, dim=1)).cpu().detach().numpy()  
        else:
            sent_embedding = torch.mean(word_embeddings, dim=1).cpu().detach().numpy()
            
        embeddings.append(sent_embedding)
    embeddings = np.vstack(embeddings)
    return embeddings

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == '__main__':
    # Enable GPU
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
    # Base parameter list for skipgram
    params_skipgram = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tokenizer': None,
                   'idf': False,
                   'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') }
    params_skipgram = dotdict(params_skipgram)
    
    # Set paths
    params_skipgram.task_path = '/home/tom/SentEval/data/' 
    params_skipgram.ckpt_path = '/home/tom/Documents/ULL/Practical 2/out/europarl/skipgram_0_True_5_1.pt'
    params_skipgram.tok_path = '/home/tom/Documents/ULL/Practical 2/data/original/europarl/training_0_True_5_1_wordIndexMap.en'
    
    # Load v_dim
    v_dim = msgpack.load(open('/home/tom/Documents/ULL/Practical 2/data/original/europarl/pad_index_skipgram.en', 'rb')) + 1
    
    # Preload model
    params_skipgram.extractor = SkipGram(v_dim, 100, v_dim-1).to(params_skipgram.device)
    
    # Whether to use tf-idf weighed embeddings
    params_skipgram.idf = True
    
    # Execute SentEval tasks
    se = senteval.engine.SE(params_skipgram, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)
    

2018-05-25 15:07:06,466 : ***** Transfer task : MR *****


2018-05-25 15:07:12,800 : Generating sentence embeddings


tensor([[ 1096]], device='cuda:0') [1096]
tensor([[ 25147]], device='cuda:0') [25147]
tensor([[ 63754]], device='cuda:0') [63754]
tensor([[ 52423,     29]], device='cuda:0') [52423, 29]
tensor([[ 22262,     29]], device='cuda:0') [22262, 29]
tensor([[ 29]], device='cuda:0') [29]
tensor([[ 78359,  20466]], device='cuda:0') [78359, 20466]
tensor([[ 4750,    29]], device='cuda:0') [4750, 29]
tensor([[ 23468,     29]], device='cuda:0') [23468, 29]
tensor([[ 62956,  13351]], device='cuda:0') [62956, 13351]
tensor([[ 35393,   2630]], device='cuda:0') [35393, 2630]
tensor([[ 11919,  29006]], device='cuda:0') [11919, 29006]
tensor([[ 2789]], device='cuda:0') [2789]
tensor([[ 36024,     29]], device='cuda:0') [36024, 29]
tensor([[ 10362,   2789]], device='cuda:0') [10362, 2789]
tensor([[ 13317,  40081,     29]], device='cuda:0') [13317, 40081, 29]
tensor([[ 13849,   6972,     29]], device='cuda:0') [13849, 6972, 29]
tensor([[ 2800,  4750,    29]], device='cuda:0') [2800, 4750, 29]
tensor([[ 241

tensor([[  9306,      2,  39853,   6972,     29]], device='cuda:0') [9306, 2, 39853, 6972, 29]
tensor([[ 133,   52,   73,   29]], device='cuda:0') [133, 52, 73, 29]
tensor([[  1786,   6972,     17,  12364,     29]], device='cuda:0') [1786, 6972, 17, 12364, 29]
tensor([[  4087,     55,  53745,     29]], device='cuda:0') [4087, 55, 53745, 29]
tensor([[ 1.0682e+05,  2.0000e+00,  1.1690e+05,  1.4024e+05,  2.9000e+01]], device='cuda:0') [106825, 2, 116903, 140242, 29]
tensor([[    55,   6600,  16255,     29]], device='cuda:0') [55, 6600, 16255, 29]
tensor([[  1782,    229,     76,  12002,     29]], device='cuda:0') [1782, 229, 76, 12002, 29]
tensor([[   984,     55,   5989,  13662,     29]], device='cuda:0') [984, 55, 5989, 13662, 29]
tensor([[    55,  53631,   1799,     29]], device='cuda:0') [55, 53631, 1799, 29]
tensor([[ 61061,     52,  18541,   5950,     29]], device='cuda:0') [61061, 52, 18541, 5950, 29]
tensor([[   998,   1047,  11550,  12411,     29]], device='cuda:0') [998, 1047, 1

tensor([[ 5.5000e+01,  5.5570e+03,  1.6300e+02,  1.3642e+05,  1.7990e+03,
          2.9000e+01]], device='cuda:0') [55, 5557, 163, 136419, 1799, 29]
tensor([[   55,  5866,    19,  8169,  1661,    29]], device='cuda:0') [55, 5866, 19, 8169, 1661, 29]
tensor([[ 5808,     2,    17,  8901,    29]], device='cuda:0') [5808, 2, 17, 8901, 29]
tensor([[    55,  34440,     19,     55,  16255,     29]], device='cuda:0') [55, 34440, 19, 55, 16255, 29]
tensor([[    55,  27186,      2,  66296,  16255,     29]], device='cuda:0') [55, 27186, 2, 66296, 16255, 29]
tensor([[    23,     55,   2111,  10405,     87,     29]], device='cuda:0') [23, 55, 2111, 10405, 87, 29]
tensor([[ 76946,  30106,     17,  53395,  30069,     29]], device='cuda:0') [76946, 30106, 17, 53395, 30069, 29]
tensor([[ 13849,  70540,     17,   1805,   2226,     29]], device='cuda:0') [13849, 70540, 17, 1805, 2226, 29]
tensor([[ 9.2004e+04,  1.2350e+03,  7.0000e+00,  1.1000e+01,  1.5488e+05,
          2.9000e+01]], device='cuda:0') [9

tensor([[ 26568,      2,  31278,      2,     17,  42596,  11055,     29]], device='cuda:0') [26568, 2, 31278, 2, 17, 42596, 11055, 29]
tensor([[ 3.6630e+03,  1.6400e+02,  1.1800e+03,  7.3300e+02,  1.1000e+01,
          1.0448e+05,  2.9000e+01]], device='cuda:0') [3663, 164, 1180, 733, 11, 104477, 29]
tensor([[   460,      0,     55,   4244,   6264,  11653,    147,     29]], device='cuda:0') [460, 0, 55, 4244, 6264, 11653, 147, 29]
tensor([[     3,  20041,    984,   1820,  41066,     14,   4765,     29]], device='cuda:0') [3, 20041, 984, 1820, 41066, 14, 4765, 29]
tensor([[    55,  24751,    528,   5397,     19,  53581,     29]], device='cuda:0') [55, 24751, 528, 5397, 19, 53581, 29]
tensor([[ 7.8300e+02,  2.3300e+02,  7.4450e+03,  2.6211e+04,  1.1000e+01,
          1.4532e+05,  1.3320e+03,  2.9000e+01]], device='cuda:0') [783, 233, 7445, 26211, 11, 145315, 1332, 29]
tensor([[  108,     2,    76,   445,   633,     9,  4112,    29]], device='cuda:0') [108, 2, 76, 445, 633, 9, 4112, 29]
t

...

tensor([[   110,    463,   1167,      2,     76,    129,    984,    187,
           1123,   2492,     29,   2365,    133,    392,    460,    517,
              0,    110,   1720,    515,     76,   3063,    110,   9579,
          19159,     29,   2874,      2,     39,      3,   1400,    763,
             29,     29,     29,     76,    129,  10476,   2789,   2789]], device='cuda:0') [110, 463, 1167, 2, 76, 129, 984, 187, 1123, 2492, 29, 2365, 133, 392, 460, 517, 0, 110, 1720, 515, 76, 3063, 110, 9579, 19159, 29, 2874, 2, 39, 3, 1400, 763, 29, 29, 29, 76, 129, 10476, 2789, 2789]
tensor([[   200,    783,  18771,    702,    210,      9,     76,    129,
             55,   8100,   1569,  11425,     17,  41066,     65,     55,
           8975,  66259,      2,     76,    498,    984,     47,     11,
          16255,    783,  18771,    376,      0,     29,     76,   7030,
              9,    532,    108,  66255,    450,    299,    257,   7191,
           6383,     29]], device='cuda:0') [200, 78

2018-05-25 15:07:24,512 : Generated sentence embeddings
2018-05-25 15:07:24,513 : Training sklearn-LogReg with (inner) 10-fold cross-validation


tensor([[ 1.8100e+02,  2.0600e+02,  1.4750e+03,  1.2900e+02,  2.0197e+04,
          2.0000e+00,  3.0825e+04,  1.2900e+02,  1.7790e+03,  2.3000e+01,
          9.3100e+02,  1.4000e+01,  4.1680e+03,  7.6000e+01,  2.0000e+00,
          7.0000e+01,  2.0000e+02,  1.1000e+01,  1.2123e+05,  5.2000e+01,
          7.3000e+01,  1.1000e+01,  8.4000e+02,  3.9790e+03,  1.5260e+03,
          2.9000e+01,  1.8100e+02,  5.5000e+01,  1.7980e+03,  1.9000e+01,
          1.1060e+03,  7.6950e+03,  2.0000e+00,  1.1000e+01,  2.9535e+04,
          1.8400e+02,  2.9378e+04,  1.7000e+01,  2.7056e+04,  3.6750e+03,
          3.0450e+03,  5.2100e+02,  1.4000e+01,  4.7000e+01,  5.7000e+03,
          2.9000e+01]], device='cuda:0') [181, 206, 1475, 129, 20197, 2, 30825, 129, 1779, 23, 931, 14, 4168, 76, 2, 70, 200, 11, 121228, 52, 73, 11, 840, 3979, 1526, 29, 181, 55, 1798, 19, 1106, 7695, 2, 11, 29535, 184, 29378, 17, 27056, 3675, 3045, 521, 14, 47, 5700, 29]
tensor([[    54,     52,    521,    110,   1476,  32028,    

2018-05-25 15:07:29,457 : Best param found at split 1: l2reg = 0.25                 with score 63.77


KeyboardInterrupt: 

In [25]:
EA_res = {'TREC': {'acc': 57.4, 'ndev': 5452, 'devacc': 53.56, 'ntest': 500}, 'SST2': {'acc': 67.11, 'ndev': 872, 'devacc': 67.2, 'ntest': 1821}, 'CR': {'acc': 70.65, 'ndev': 3775, 'devacc': 70.64, 'ntest': 3775}, 'MPQA': {'acc': 83.78, 'ndev': 10606, 'devacc': 83.89, 'ntest': 10606}, 'MR': {'acc': 64.72, 'ndev': 10662, 'devacc': 64.53, 'ntest': 10662}, 'STS14': {'headlines': {'pearson': (0.5781994919225272, 4.020000246168714e-68), 'nsamples': 750, 'spearman': 'SpearmanrResult(correlation=0.5705232857978165, pvalue=5.589791453157285e-66)'}, 'deft-news': {'pearson': (0.6220449533709097, 1.6081763124275315e-33), 'nsamples': 300, 'spearman': 'SpearmanrResult(correlation=0.578333454614915, pvalue=3.56018796171822e-28)'}, 'images': {'pearson': (0.6578237117176955, 3.654157740608177e-94), 'nsamples': 750, 'spearman': 'SpearmanrResult(correlation=0.6438621123340441, pvalue=4.9441358581162626e-89)'}, 'OnWN': {'pearson': (0.658846051664604, 1.5004930667755487e-94), 'nsamples': 750, 'spearman': 'SpearmanrResult(correlation=0.7165302641879749, pvalue=4.038459846586195e-119)'}, 'tweet-news': {'pearson': (0.6233064013917692, 6.006549841321549e-82), 'nsamples': 750, 'spearman': 'SpearmanrResult(correlation=0.552714320253843, pvalue=3.22156860724007e-61)'}, 'all': {'pearson': {'mean': 0.5813380714320986, 'wmean': 0.5951356658320022}, 'spearman': {'mean': 0.5708291603366306, 'wmean': 0.5865540558636717}}, 'deft-forum': {'pearson': (0.3478078185250862, 3.053271390924939e-14), 'nsamples': 450, 'spearman': 'SpearmanrResult(correlation=0.3630115248311902, pvalue=1.833847351421204e-15)'}}, 'SICKEntailment': {'acc': 74.75, 'ndev': 500, 'devacc': 72.6, 'ntest': 4927}, 'MRPC': {'acc': 70.96, 'f1': 80.1, 'devacc': 70.61, 'ndev': 4076, 'ntest': 1725}, 'SUBJ': {'acc': 79.15, 'ndev': 10000, 'devacc': 79.17, 'ntest': 10000}}
SG_avg_res = {'SUBJ': {'acc': 79.83, 'devacc': 79.93, 'ntest': 10000, 'ndev': 10000}, 'MR': {'acc': 64.91, 'devacc': 65.06, 'ntest': 10662, 'ndev': 10662}, 'CR': {'acc': 70.6, 'devacc': 70.57, 'ntest': 3775, 'ndev': 3775}, 'STS14': {'deft-forum': {'spearman': 'SpearmanrResult(correlation=0.30898757056284787, pvalue=2.0691854690433753e-11)', 'pearson': (0.27256288243965693, 4.1651890020195675e-09), 'nsamples': 450}, 'images': {'spearman': 'SpearmanrResult(correlation=0.4408194211200905, pvalue=5.280384646229476e-37)', 'pearson': (0.4117261507554508, 4.7518699650716877e-32), 'nsamples': 750}, 'deft-news': {'spearman': 'SpearmanrResult(correlation=0.5647511600041103, pvalue=1.13194354561933e-26)', 'pearson': (0.5422089540432408, 2.509906356244403e-24), 'nsamples': 300}, 'all': {'spearman': {'mean': 0.48469429259169844, 'wmean': 0.48914400626451704}, 'pearson': {'mean': 0.45847437877611935, 'wmean': 0.4632991494509817}}, 'OnWN': {'spearman': 'SpearmanrResult(correlation=0.5986940260146392, pvalue=3.933092783581038e-74)', 'pearson': (0.5214190523169768, 1.6237220055815684e-53), 'nsamples': 750}, 'tweet-news': {'spearman': 'SpearmanrResult(correlation=0.5375884592346126, pvalue=2.147350633544745e-57)', 'pearson': (0.5341431243615529, 1.4997705284872292e-56), 'nsamples': 750}, 'headlines': {'spearman': 'SpearmanrResult(correlation=0.4573251186138901, pvalue=4.902572881630306e-40)', 'pearson': (0.4687861087398376, 3.055636088817881e-42), 'nsamples': 750}}, 'SST2': {'acc': 64.31, 'devacc': 63.99, 'ntest': 1821, 'ndev': 872}, 'TREC': {'acc': 53.8, 'devacc': 53.52, 'ntest': 500, 'ndev': 5452}, 'SICKEntailment': {'acc': 67.73, 'devacc': 69.6, 'ntest': 4927, 'ndev': 500}, 'MPQA': {'acc': 83.59, 'devacc': 83.62, 'ntest': 10606, 'ndev': 10606}, 'MRPC': {'acc': 70.14, 'f1': 80.23, 'devacc': 70.56, 'ntest': 1725, 'ndev': 4076}}
SG_tfidf_res = {'SUBJ': {'acc': 77.51, 'devacc': 77.53, 'ntest': 10000, 'ndev': 10000}, 'MR': {'acc': 64.24, 'devacc': 64.07, 'ntest': 10662, 'ndev': 10662}, 'CR': {'acc': 69.22, 'devacc': 69.41, 'ntest': 3775, 'ndev': 3775}, 'STS14': {'deft-forum': {'spearman': 'SpearmanrResult(correlation=0.316315651904115, pvalue=6.4858192321523515e-12)', 'pearson': (0.276666708551114, 2.378646491257731e-09), 'nsamples': 450}, 'images': {'spearman': 'SpearmanrResult(correlation=0.45347644118861796, pvalue=2.5846122070851603e-39)', 'pearson': (0.4355811792783599, 4.473764339992488e-36), 'nsamples': 750}, 'deft-news': {'spearman': 'SpearmanrResult(correlation=0.5484910896581424, pvalue=5.803318096557867e-25)', 'pearson': (0.5601467570097868, 3.528331007610503e-26), 'nsamples': 300}, 'all': {'spearman': {'mean': 0.5085991098439903, 'wmean': 0.519194748901482}, 'pearson': {'mean': 0.49753726563574924, 'wmean': 0.5076937712376356}}, 'OnWN': {'spearman': 'SpearmanrResult(correlation=0.7023028905428166, pvalue=1.6489620448032347e-112)', 'pearson': (0.6749156474824787, 7.848620112501906e-101), 'nsamples': 750}, 'tweet-news': {'spearman': 'SpearmanrResult(correlation=0.5582633328557121, pvalue=1.1364337930928656e-62)', 'pearson': (0.5582010433457185, 1.1803246262212194e-62), 'nsamples': 750}, 'headlines': {'spearman': 'SpearmanrResult(correlation=0.4727452529145376, pvalue=5.053919999036877e-43)', 'pearson': (0.4797122581470376, 2.010716425101528e-44), 'nsamples': 750}}, 'SST2': {'acc': 64.8, 'devacc': 64.22, 'ntest': 1821, 'ndev': 872}, 'TREC': {'acc': 48.8, 'devacc': 48.79, 'ntest': 500, 'ndev': 5452}, 'SICKEntailment': {'acc': 64.52, 'devacc': 63.8, 'ntest': 4927, 'ndev': 500}, 'MPQA': {'acc': 83.28, 'devacc': 83.47, 'ntest': 10606, 'ndev': 10606}, 'MRPC': {'acc': 69.68, 'f1': 80.08, 'devacc': 70.46, 'ntest': 1725, 'ndev': 4076}}

for res in [EA_res, SG_avg_res, SG_tfidf_res]:
    for key, value in res.items():
        if key == "STS14":
            print('{}: {}'.format(key, value['all']))
        else:
            print('{}: {}'.format(key, value))
        print('___________________________')
    print("\n-----------------------------------------\n")

SUBJ: {'acc': 79.15, 'devacc': 79.17, 'ntest': 10000, 'ndev': 10000}
___________________________
MR: {'acc': 64.72, 'devacc': 64.53, 'ntest': 10662, 'ndev': 10662}
___________________________
CR: {'acc': 70.65, 'devacc': 70.64, 'ntest': 3775, 'ndev': 3775}
___________________________
STS14: {'spearman': {'mean': 0.5708291603366306, 'wmean': 0.5865540558636717}, 'pearson': {'mean': 0.5813380714320986, 'wmean': 0.5951356658320022}}
___________________________
SST2: {'acc': 67.11, 'devacc': 67.2, 'ntest': 1821, 'ndev': 872}
___________________________
TREC: {'acc': 57.4, 'devacc': 53.56, 'ntest': 500, 'ndev': 5452}
___________________________
SICKEntailment: {'acc': 74.75, 'devacc': 72.6, 'ntest': 4927, 'ndev': 500}
___________________________
MPQA: {'acc': 83.78, 'devacc': 83.89, 'ntest': 10606, 'ndev': 10606}
___________________________
MRPC: {'acc': 70.96, 'f1': 80.1, 'devacc': 70.61, 'ntest': 1725, 'ndev': 4076}
___________________________

-----------------------------------------

S